In [1]:
#Dependencies
from bs4 import BeautifulSoup
import pandas as pd
from splinter import Browser
import datetime as datetime
import time
import json
import re

In [3]:
#create splinter browser
executable_path = {'executable_path':"../chromedriver.exe"}
browser = Browser('chrome', **executable_path, headless=False)

In [4]:

#define url as variable and open with browser
url = "https://dallasites101.com/archive/2020-dallas-restaurant-openings-closings-and-coming-soons"
browser.visit(url)
time.sleep(1)
html = browser.html
soup = BeautifulSoup(html, 'lxml')

In [58]:
#Grab lists of openings and closings
listings = soup.find_all('ul')

In [91]:
# List order: 0=Rest Open, 1= Rest Soon, 2= Rest Closed, 3 = Bar Open, 4 = Bar Soon,
            # 5 = Bar Closed, 6 = Sweets Open, 7 = Sweets Soon, 8= Sweets Closed, 
            # 9 = Cafe Open, 10 = Cafe Soon, 11 = Cafe Closed
test =listings[0].find_all('li')
test1 = test[19].find('a').get('href')
test1

'https://www.duggburger.com'

In [101]:
name =[]
neighborhood = []
kind = []
status = []
hyperlink = []
date = []

for listing in listings[0].find_all('li'):
    temp = listing.find('p')
    temptext = temp.text.replace("\xa0", " ")
    temptext = temptext.split(' | ')
    name.append(temptext[0])
    neighborhood.append(temptext[1])
    kind.append('Restaurant')
    status.append('Open')
    try:
        templink = listing.find('a').get('href')
        hyperlink.append(templink)
    except AttributeError:
        hyperlink.append(None)
    date.append(datetime.datetime.today().strftime("%m/%d/%y"))


['12/28/20', '12/28/20', '12/28/20', '12/28/20', '12/28/20', '12/28/20', '12/28/20', '12/28/20', '12/28/20', '12/28/20', '12/28/20', '12/28/20', '12/28/20', '12/28/20', '12/28/20', '12/28/20', '12/28/20', '12/28/20', '12/28/20', '12/28/20', '12/28/20', '12/28/20', '12/28/20', '12/28/20', '12/28/20', '12/28/20', '12/28/20', '12/28/20', '12/28/20', '12/28/20', '12/28/20', '12/28/20', '12/28/20', '12/28/20', '12/28/20', '12/28/20', '12/28/20', '12/28/20', '12/28/20', '12/28/20', '12/28/20', '12/28/20', '12/28/20', '12/28/20', '12/28/20', '12/28/20', '12/28/20', '12/28/20', '12/28/20', '12/28/20', '12/28/20', '12/28/20', '12/28/20', '12/28/20', '12/28/20', '12/28/20', '12/28/20', '12/28/20', '12/28/20', '12/28/20', '12/28/20', '12/28/20', '12/28/20', '12/28/20', '12/28/20', '12/28/20', '12/28/20', '12/28/20', '12/28/20', '12/28/20', '12/28/20', '12/28/20', '12/28/20', '12/28/20', '12/28/20', '12/28/20', '12/28/20', '12/28/20', '12/28/20', '12/28/20', '12/28/20', '12/28/20', '12/28/20', '12

In [23]:
test[1].find('p').text

'A'

In [37]:
len(listings)

12

In [41]:
listings[11]

<ul data-rte-list="default"><li><p class="" style="white-space:pre-wrap;">Good Companions | <em>Bishop Arts</em></p></li><li><p class="" style="white-space:pre-wrap;">StellaNova | <em>Park Cities</em></p></li></ul>